# K近邻算法
+ 既可以是分类（类别），也可以是回归（具体值）

### Kmeans（K均值）与KNN（K近邻）的区别
Kmeans是把数据分成K类，KNN是把K个相邻近的数据归为一组
+ Kmeans它把n个对象根据他们的属性分为k个聚类以便使得所获得的聚类满足：同一聚类中的对象相似度较高；而不同聚类中的对象相似度较小
+ KNN如果一个样本在特征空间中的k个最相似(即特征空间中最邻近)的样本中的大多数属于某一个类别，则该样本也属于这个类别。该方法在定类决策上只依据最邻近的一个或者几个样本的类别来决定待分样本所属的类别

### 训练集与测试集
+ 训练集是用来训练模型的，也就是用训练集中的数据通过计算得出具体模型
+ 测试集是用来验证模型的，也就是验证训练集中的得出模型的准确度

###  机器学习最常用的库 sklearn
其中数据标准化，调用StandardScaler  
StandardScaler().fit_transform(data)

### scipy是统计用的包，其中有现成的统计公式

### 数据结构--数据框dataframe
+ data.head()  #返回data的前几行数据，默认为前五行，需要前十行则data.head(10) 
+ data.tail()  #返回data的后几行数据，默认为后五行，需要后十行则data.tail(10) 

### Pandas库中的sample--随机抽样
DataFrame.sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None)
+ n：要抽取的行数
+ frac：抽取行的比例，例如frac=0.8，就是抽取其中80%
+ replace：是否为有放回抽样，True:有放回抽样，False:未放回抽样
+ weights：字符索引或概率数组，xis=0:为行字符索引或概率数组，axis=1:为列字符索引或概率数组
+ random_state：int: 随机数发生器种子，或numpy.random.RandomState。random_state=None,取得数据不重复，random_state=1,可以取得重复数据
+ axis：选择抽取数据的行还是列，axis=0:抽取行，axis=1:抽取列

### 替换replace
str.replace(old, new[, max]) 

参数解读:
+ old -- 将被替换的子字符串。
+ new -- 新字符串，用于替换old子字符串。
+ max -- 可选字符串, 替换不超过 max 次
 
替换字符含义：
+ ^ 表示以什么开头
+ \s 代表匹配一些空白符     包括：空格、Tab、换行、回车,等价于 [\t\r\n ]
+ *表示有一个或者多个
+ | 表示或者
+ $ 代表以什么结尾

---------------------------------------------------------------------------------------------------------------------------------------------

## Airbnb 房价预测任务
## 导入数据

In [2]:
import pandas as pd

# 选取数据的字段
features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews'] 

# 读取数据
dc_listings = pd.read_csv(r'D:/001  学习文件/001  学习笔记/020  唐宇迪机器学习实战_20200326/机器学习文件/006 K近邻实例/KNN/listings.csv') 

# 数据框取值
dc_listings = dc_listings[features] 
# 数据行列数
print(dc_listings.shape) 

# 查看对象的前n行数据
dc_listings.head()

(3723, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,$160.00,1,1125,0
1,6,3.0,3.0,3.0,$350.00,2,30,65
2,1,1.0,2.0,1.0,$50.00,2,1125,1
3,2,1.0,1.0,1.0,$95.00,1,1125,0
4,4,1.0,1.0,1.0,$50.00,7,1125,0


#### 数据特征：

* accommodates: 可以容纳的旅客
* bedrooms: 卧室的数量
* bathrooms: 厕所的数量
* beds: 床的数量
* price: 每晚的费用
* minimum_nights: 客人最少租了几天
* maximum_nights: 客人最多租了几天
* number_of_reviews: 评论的数量

## K近邻原理

### 如果我有一个3个房间的房子，我能租多少钱呢？
讲道理，咱是不是得看一看3个房间的别人都租到多少钱啊？--货比三家

K代表我们的候选对象个数，也就是我和我房间数量最相近的其他房子的价格

### 疑问：为什么取的是最大的3个？？？--似乎是随机选取

## 距离的定义
如何让才能知道哪些数据样本跟我最相近呢？   
通过[欧式距离](https://baike.baidu.com/item/%E6%AC%A7%E5%87%A0%E9%87%8C%E5%BE%97%E5%BA%A6%E9%87%8F/1274107?fromtitle=%E6%AC%A7%E5%BC%8F%E8%B7%9D%E7%A6%BB&fromid=2809635&fr=aladdin) 进行计算，详细公式如下： 

![6.png](https://i.loli.net/2020/03/28/1PniOKx8lY2EDkL.png)

其中Q1到Qn是一条数据的所有特征信息，P1到Pn是另一条数据的所有特征信息

假设我的房间数量是3个：

In [3]:
import numpy as np

our_acc_value = 3

# 用所有数据房间数减去3，然后结果取绝对值
dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)

# 按差值分组统计，并排序
dc_listings.distance.value_counts().sort_index()

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

这里我们只用了绝对值来计算，和我们距离为0的（同样数量的房间）有461个

#### sample操作可以得到洗牌后数据--随机抽样

In [4]:
# 随机打乱数据
dc_listings = dc_listings.sample(frac=1,random_state=0) 

# 按距离排序
dc_listings = dc_listings.sort_values('distance')

# 显示前5行
dc_listings.price.head()

2645     $75.00
2825    $120.00
2145     $90.00
2541     $50.00
3349    $105.00
Name: price, dtype: object

需要注意的是，这里面的数据现在是字符串，如果想进行计算，需要转换一下。

In [5]:
# 替换“$”或者，为空
dc_listings['price'] = dc_listings.price.str.replace("\$|,","").astype(float)

# 取前5的数据的平均值
mean_price = dc_listings.price.iloc[:5].mean()

mean_price

88.0

得到了平均价格，也就是我们房子的大致价格。

## 模型评估

![7.png](https://i.loli.net/2020/03/28/OboRu7gJPaHxdMm.png)

首先制定好训练集和测试集，训练集训练模型，出现的模型尽量贴合训练数据；测试集是新的数据，来衡量模型的结果，小心模型的过拟合

In [6]:
# 删除之前计算的“距离”指标
dc_listings.drop("distance",axis=1)

# 打乱数据顺序
dc_listings = dc_listings.sample(frac=1,random_state=0) 

# 切分训练集和测试集
train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

基于单变量预测价格

In [13]:
# 定义预测房间的函数
def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    # new_listing_value 来自哪里？用表中的每一个数据与其他数据进行比较
    temp_df['distance'] = np.abs(dc_listings[feature_column]-new_listing_value) 
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

In [14]:
## 计算测试集的预测价格
test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')

这样就得到测试集中所有房子的价格，然后通过RMSE进行预测值与实际值差距的检验  
##### RMSE(root mean squared error) 均方根误差

![8.png](https://i.loli.net/2020/03/28/Cn38JUxdoPG9m2s.png)

In [18]:
## 计算测试集均方根误差
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse**(1/2)
rmse

212.98927967051543

现在就得到了一个变量模型的评估得分

## 不同的变量效果会不会不同呢？

数据预处理会涉及：标准化或者归一化。        
机器学习最常用的是sklearn

In [25]:
# 循环不同变量的均方根误差（REMS）
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    test_df['predicted_price'] = test_df[feature].apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse**(1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: 212.98927967051543
RMSE for the bedrooms column: 199.80935328065033
RMSE for the bathrooms column: 230.24716705684227
RMSE for the number_of_reviews column: 235.91327066995507


看起来结果之间还是有差异的，但是“bedrooms”的均方跟最小，也就是用该变量预测的结果误差最小，接着利用综合信息来进行预测，因为变量之间单位不同，而且量级差异也较大，所以在进行计算时，需先进行标准化或者归一化

In [7]:
import pandas as pd
# preprocessing 数据预处理模块
from sklearn.preprocessing import StandardScaler
features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews'] 

# 读取数据
dc_listings = pd.read_csv(r'D:/001  学习文件/001  学习笔记/020  唐宇迪机器学习实战_20200326/机器学习文件/006 K近邻实例/KNN/listings.csv') 

# 数据框取值
dc_listings = dc_listings[features] 

# 替换“$”或者，为空
dc_listings['price'] = dc_listings.price.str.replace("\$|,","").astype(float)

# 删除空白数据行
dc_listings = dc_listings.dropna()

# 数据标准化
# dc_listings[features] = StandardScaler().fit_transform(dc_listings[features])
dc_listings[features] = StandardScaler().fit_transform(dc_listings[features])

normalized_listings = dc_listings

print(normalized_listings.shape)

normalized_listings.head()

(3671, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,0.401420,-0.249501,-0.439211,0.297386,0.081119,-0.341421,-0.016575,-0.516779
1,1.399466,2.129508,2.969551,1.141704,1.462622,-0.065047,-0.016606,1.706767
2,-1.095648,-0.249501,1.265170,-0.546933,-0.718699,-0.065047,-0.016575,-0.482571
3,-0.596625,-0.249501,-0.439211,-0.546933,-0.391501,-0.341421,-0.016575,-0.516779
4,0.401420,-0.249501,-0.439211,-0.546933,-0.718699,1.316824,-0.016575,-0.516779


In [8]:
# 划分训练集与测试集
norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

多变量之间的距离计算，仍然采用欧式距离

![9.png](https://i.loli.net/2020/03/28/f5p7ZK8q3HugePF.png)

In [9]:
# scipy是统计用的包，其中有现成的统计公式
# 选择两组数据进行测试计算欧式距离
from scipy.spatial import distance

first_listing = normalized_listings.iloc[0][['accommodates','bathrooms']]
fifth_listing = normalized_listings.iloc[20][['accommodates','bathrooms']]
first_fifth_distance = distance.euclidean(first_listing,fifth_listing)

first_fifth_distance

3.723019604017032

## 多变量KNN模型

In [10]:
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

cols = ['accommodates','bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns = cols,axis=1)
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**2
mse = norm_test_df['squared_error'].mean()
rmse = mse**(1/2)
print(rmse)

0.7894063922577537


## 使用Sklearn 来完成KNN

In [13]:
from sklearn.neighbors import KNeighborsRegressor
cols = ['accommodates','bathrooms']

# 实例化KNN算法函数,()中默认k = 5 
knn = KNeighborsRegressor()

knn.fit(norm_train_df[cols],norm_train_df['price'])
two_features_predictions = knn.predict(norm_test_df[cols])

In [15]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error(norm_test_df['price'], two_features_predictions)
two_features_rmse = two_features_mse**(1/2)
print(two_features_rmse)

0.857101359198754


#### 加入更多特征

In [16]:
knn = KNeighborsRegressor()

cols = ['accommodates','bedrooms','bathrooms','beds','minimum_nights','maximum_nights','number_of_reviews']

knn.fit(norm_train_df[cols], norm_train_df['price'])
four_features_predictions = knn.predict(norm_test_df[cols])
four_features_mse = mean_squared_error(norm_test_df['price'], four_features_predictions)
four_features_rmse = four_features_mse ** (1/2)
four_features_rmse

0.8243838530880285